In [185]:
import pandas as pd 
import numpy as np 
import re 

In [186]:
file=r'Best_sell_phone_raw.csv'
df=pd.read_csv(file)

# Clean raw Data

In [187]:
list_na=dict(df.isna().sum())
for col,count in list_na.items():
    if count > 0:
        df.loc[df[col].isna(),col] = 'NaN'

In [188]:
def clean_utility(row):
    return ' '.join(row.replace('\n','').strip(' ').split())
df['phone_utility']=df['phone_utility'].apply(clean_utility)

def clean_screen(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_screen']=df['phone_screen'].apply(clean_screen)

def clean_rating(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_rating']=df['phone_rating'].apply(clean_rating)

def enrich_vote_rating_raw(row):
    _=row.split()[0]
    return _ 
df['vote_rating'] = df['phone_rating'].apply(enrich_vote_rating_raw)

def enrich_sold_quantity_raw(row):
    _=row.split()
    if len(_) > 3:
        raw_sold_quantity=_[4]
        return raw_sold_quantity
    else:
        return 0
df['sold_quantity'] = df['phone_rating'].apply(enrich_sold_quantity_raw)

df.loc[df['vote_rating'].str.isalpha(),'vote_rating'] = 0

def enrich_sold_quantity(row):
    if str(row).find('k') > 0:
        _ = int(row.replace(',','').replace('k','')) * 1000
        return _
    else:
        return row
df['sold_quantity']=df['sold_quantity'].apply(enrich_sold_quantity)

def clean_phone_screen(row):
    return row.replace('"','').strip()
df['phone_screen']=df['phone_screen'].apply(clean_phone_screen)

# Transfrom Datatype

In [189]:
df['phone_price']=df['phone_price'].astype('int64')
df['sold_quantity']=df['sold_quantity'].astype('int64')
df['vote_rating']=df['vote_rating'].astype('float64')

# Remove oulier

In [190]:
df_2=df.copy()
df_2=df_2[df_2['sold_quantity']>0]

# Enrich Data

In [191]:
df_2['gross_revenue']=df_2['sold_quantity'] * df_2['phone_price']

In [192]:
def price_segment(row):
    if row < 10000000:
        return '0 to 10M VND'
    elif row < 20000000:
        return '10M to 20M VND'
    else:
        return 'Upper 20MVND'
df_2['price_segment']=df_2['phone_price'].apply(price_segment)

In [193]:
def return_size_screen_phone(row):
    list_row=row.split()
    leght_list=len(list_row)
    if leght_list < 2:
        return ' '.join(list_row)
    elif leght_list < 5:
        return list_row[-1]
    else:
        return ' '.join(list_row[list_row.index('Chính'):])
df_2['size_screen_inch']=df_2['phone_screen'].apply(return_size_screen_phone)

In [194]:
def return_screen_phone(row):
    list_row=row.split()
    leght_list=len(list_row)
    if leght_list < 2:
        return ' '.join(list_row)
    elif leght_list < 5:
        return ' '.join(list_row[:-1])
    else:
        return ' '.join(list_row[:list_row.index('Chính')])
def detail_none_screen(row):
    _ = row.replace('.','')
    if _.isdigit():
        return 'NaN'
    else:
        return row
df_2['info_screen']=df_2['phone_screen'].apply(return_screen_phone).apply(detail_none_screen)

In [195]:
def rating_segment(row):
    if 0 < row < 3:
        return 'Poor'
    elif 3 < row < 4.5:
        return 'Average'
    else:
        return 'Excellent'
df_2['rating_segment'] = df_2['vote_rating'].apply(rating_segment)

In [196]:
def return_ram(row):
    list_row=row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        ram_idx=list_row.index('RAM:')
        return list_row[ram_idx+1]
    else:
        return 'NaN'
def return_rom(row):
    list_row=row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        rom_idx=list_row.index('lượng:')
        return list_row[rom_idx+1]
    else:
        return 'NaN'
df_2['RAM(GB)'] = df_2['phone_utility'].apply(return_ram)
df_2['ROM(GB)'] = df_2['phone_utility'].apply(return_rom)

In [197]:
def return_battery(row):
    list_row=row.split()
    if 'Pin' in list_row:
        pin_idx=list_row.index('Pin')
        return ' '.join(list_row[pin_idx+1:pin_idx+3]).replace(',','')
    else:
        return 'NaN'
df_2['battery']=df_2['phone_utility'].apply(return_battery)

In [198]:
df_2['phone_price_Milion_VND'] = df_2['phone_price'] / 1000000

In [199]:
df_2['gross_revenue_Bilion_VND'] = df_2['gross_revenue'] / 1000000000

In [200]:
df_2

,phone_id,phone_name,phone_price,phone_brand,category,phone_img,phone_screen,phone_utility,phone_rating,vote_rating,...,gross_revenue,price_segment,size_screen_inch,info_screen,rating_segment,RAM(GB),ROM(GB),battery,phone_price_Milion_VND,gross_revenue_Bilion_VND
1,334323,Điện thoại OPPO A5 Pro 8GB/256GB,6990000,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/334323/...,HD+ 6.67,Chip Snapdragon 6s Gen 1 8 nhân RAM: 8 GB Dung...,5 • Đã bán 184 So sánh,5.0,...,1286160000,0 to 10M VND,6.67,HD+,Excellent,8,256,5800 mAh,6.99,1.28616
3,329149,Điện thoại iPhone 16 Pro Max 256GB,31390000,iPhone (Apple),Điện thoại,https://cdn.tgdd.vn/Products/Images/42/329149/...,Super Retina XDR 6.9,Chip Apple A18 Pro 6 nhân RAM: 8 GB Dung lượng...,"4.9 • Đã bán 115,8k So sánh",4.9,...,36349620000000,Upper 20MVND,6.9,Super Retina XDR,Excellent,8,256,33 giờ,31.39,36349.62000
5,333347,Điện thoại Samsung Galaxy S25 Ultra 5G 12GB/256GB,28490000,Samsung,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/333347/...,Quad HD+ (2K+) 6.9,Chip Qualcomm Snapdragon 8 Elite For Galaxy 8 ...,"4.9 • Đã bán 2,2k So sánh",4.9,...,626780000000,Upper 20MVND,6.9,Quad HD+ (2K+),Excellent,12,256,5000 mAh,28.49,626.78000
6,333147,Điện thoại Xiaomi Redmi Note 14 8GB/128GB,5490000,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/333147/...,Full HD+ 6.67,Chip MediaTek Helio G99-Ultra 8 nhân RAM: 8 GB...,"4.9 • Đã bán 48,5k So sánh",4.9,...,2662650000000,0 to 10M VND,6.67,Full HD+,Excellent,8,128,5500 mAh,5.49,2662.65000
7,320730,Điện thoại Xiaomi Redmi Note 14 Pro 8GB/256GB,7390000,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/320730/...,Full HD+ 6.67,Chip MediaTek Helio G100-Ultra 8 nhân RAM: 8 G...,"4.9 • Đã bán 14,4k So sánh",4.9,...,1064160000000,0 to 10M VND,6.67,Full HD+,Excellent,8,256,5500 mAh,7.39,1064.16000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,303833,Điện thoại iPhone 15 Pro 1TB,35490000,iPhone (Apple),Điện thoại,https://cdn.tgdd.vn/Products/Images/42/303833/...,Super Retina XDR 6.1,Chip Apple A17 Pro 6 nhân RAM: 8 GB Dung lượng...,5 • Đã bán 88 So sánh,5.0,...,3123120000,Upper 20MVND,6.1,Super Retina XDR,Excellent,8,1,3274 mAh,35.49,3.12312
109,323000,Điện thoại realme C65 8GB/128GB,4290000,realme,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/323000/...,HD+ 6.67,Chip MediaTek Helio G85 RAM: 8 GB Dung lượng: ...,"4.9 • Đã bán 14,3k So sánh",4.9,...,613470000000,0 to 10M VND,6.67,HD+,Excellent,8,128,5000 mAh,4.29,613.47000
110,332255,Điện thoại HONOR Magic V3 5G 12GB/512GB,37990000,HONOR,Điện thoại,NaN,Quad HD (2K) Chính 7.92 & Phụ 6.43,Chip Snapdragon 8 Gen 3 8 nhân RAM: 12 GB Dung...,5 • Đã bán 75 So sánh,5.0,...,2849250000,Upper 20MVND,Chính 7.92 & Phụ 6.43,Quad HD (2K),Excellent,12,512,5150 mAh,37.99,2.84925
111,311354,Điện thoại OPPO A58 8GB/128GB,4790000,OPPO,Điện thoại,NaN,Full HD+ 6.72,Chip MediaTek Helio G85 RAM: 8 GB Dung lượng: ...,"4.9 • Đã bán 123,9k So sánh",4.9,...,5934810000000,0 to 10M VND,6.72,Full HD+,Excellent,8,128,5000 mAh,4.79,5934.81000


In [201]:
df_3=df_2.drop(['phone_rating','phone_screen','category','phone_id'],axis=1)

In [202]:
df_3=df_3[['phone_name','phone_brand','phone_price','phone_price_Milion_VND','phone_img','phone_utility','RAM(GB)','ROM(GB)','battery','info_screen','size_screen_inch','vote_rating','price_segment','rating_segment','sold_quantity','gross_revenue','gross_revenue_Bilion_VND']]

In [203]:
df_3.to_csv('Best_sell_phone_cleaned.csv',index=False)

In [204]:
df_3.columns

Index(['phone_name', 'phone_brand', 'phone_price', 'phone_price_Milion_VND',
       'phone_img', 'phone_utility', 'RAM(GB)', 'ROM(GB)', 'battery',
       'info_screen', 'size_screen_inch', 'vote_rating', 'price_segment',
       'rating_segment', 'sold_quantity', 'gross_revenue',
       'gross_revenue_Bilion_VND'],
      dtype='object')